# Контест отборочного этапа ИИ I_profi

### Task 1 "Фруктовая метрика"

In [1]:
import numpy as np


# Данные задачи
data = [
    {"probability": 0.85, "label": "Спелый"},
    {"probability": 0.55, "label": "Спелый"},
    {"probability": 0.65, "label": "Неспелый"},
    {"probability": 0.40, "label": "Спелый"},
    {"probability": 0.95, "label": "Спелый"},
    {"probability": 0.75, "label": "Неспелый"},
    {"probability": 0.50, "label": "Спелый"},
    {"probability": 0.60, "label": "Спелый"},
    {"probability": 0.30, "label": "Неспелый"},
    {"probability": 0.80, "label": "Спелый"},
]

# Конвертируем метки в бинарный формат (1 = Спелый, 0 = Неспелый)
probabilities = np.array([item["probability"] for item in data])
labels = np.array([1 if item["label"] == "Спелый" else 0 for item in data])

# Пороговые значения
thresholds = np.round(np.arange(0, 1., 0.1), 5)
precision_values = []

# Вычисление Precision для каждого порога
for t in thresholds:
    predictions = probabilities >= t
    TP = np.sum((predictions == 1) & (labels == 1))  # Истинно положительные
    FP = np.sum((predictions == 1) & (labels == 0))  # Ложно положительные
    precision = TP / (TP + FP)
    precision_values.append(precision)

# Вычисление Average Precision
average_precision = np.mean(precision_values)

print(average_precision)


0.7744444444444444


### Task 2 Разделяющий эллипс

In [7]:
import numpy as np

# Данные задачи
mu1 = np.array([-2, 3])
mu2 = np.array([1, 0])
Sigma1 = np.array([[2, 0], [0, 1]])
Sigma2 = np.array([[3, -1], [-1, 4]])

# Разница средних
delta_mu = mu2 - mu1

# Инвертируем ковариационные матрицы
Sigma1_inv = np.linalg.inv(Sigma1)

Sigma2_inv = np.linalg.inv(Sigma2)

# Матрица, описывающая эллипс
A = Sigma1_inv - Sigma2_inv
B = 2 * (Sigma2_inv @ mu2 - Sigma1_inv @ mu1)
C = mu1.T @ Sigma1_inv @ mu1 - mu2.T @ Sigma2_inv @ mu2

# Найдём собственные значения и векторы матрицы A
eigvals, eigvecs = np.linalg.eig(A)

# Длина большой полуоси эллипса: 1 / sqrt(минимальное собственное значение)
# (так как уравнение эллипса имеет вид x^T A x + ... = const, а масштаб осей обратно пропорционален sqrt(eigenvalue))
max_semi_axis = 1 / np.sqrt(np.min(np.abs(eigvals)))


round(max_semi_axis)

3

### Task 3  Двойное блуждание по числовой прямой

In [8]:
import numpy as np

# Параметры задачи
steps = 100
simulations = 1_000_000
start_distance = 10

# Симуляции
meeting_count = 0

for _ in range(simulations):
    # Генерация случайных блужданий для обоих друзей
    friend1 = np.cumsum(np.random.choice([-1, 1], steps))
    friend2 = np.cumsum(np.random.choice([-1, 1], steps)) + start_distance
    
    # Проверка на совпадение координат
    if np.any(friend1 == friend2):
        meeting_count += 1

# Вычисление вероятности
probability = meeting_count / simulations
round(probability, 3)


0.48

### Task 4 Минимизация MSE

In [10]:
from fractions import Fraction

# Ожидания для различных степеней X
E_X2 = Fraction(1, 3)
E_X3 = Fraction(1, 4)
E_X4 = Fraction(1, 5)
E_X5 = Fraction(1, 6)
E_X6 = Fraction(1, 7)

# Коэффициенты
a1 = Fraction(-2, 5)
a2 = Fraction(4, 3)

# Формула для MSE
MSE = (
    E_X6
    - 2 * a1 * E_X4
    - 2 * a2 * E_X5
    + a1**2 * E_X2
    + a2**2 * E_X4
    + 2 * a1 * a2 * E_X3
)

print(MSE)

1/1575


### Task 5 Комбинации для геймера

In [11]:
def minimal_code_length(n):
    total_length = 0
    level = 0
    
    while n > 0:
        # На текущем уровне двоичного дерева количество кодов ограничено 2^level
        codes_at_level = min(n, 2**level)
        total_length += codes_at_level * (level + 1)
        n -= codes_at_level
        level += 1
    
    return total_length

n = 100
result = minimal_code_length(n)
print(result)


580


### Task 6 Квантизация распределения Лапласа

In [9]:
import numpy as np
from scipy.integrate import quad
from scipy.optimize import minimize

# Ввод параметров
l = float(input())
x0, x1 = map(float, input().split(","))


# Функция плотности распределения Лапласа
def P(x):
    return 1 / (2 * l) * np.exp(-np.abs(x) / l)


# Центроид интервала
def centroid(a, b):
    numerator, _ = quad(lambda x: x * P(x), a, b)
    denominator, _ = quad(P, a, b)
    return numerator / denominator if denominator != 0 else 0


# Функция для вычисления корреляции
def correlation(y_values):
    y2, y1, y0 = y_values
    intervals = [
        (x1, np.inf, y2),
        (x0, x1, y1),
        (-x0, x0, y0),
        (-x1, -x0, -y1),
        (-np.inf, -x1, -y2),
    ]

    for a, b, y in intervals:
        prob, _ = quad(P, a, b)
        numerator += y * quad(lambda x: x * P(x), a, b)[0]
        denominator_x += quad(lambda x: x**2 * P(x), a, b)[0]
        denominator_y += y**2 * prob

    if denominator_x == 0 or denominator_y == 0:
        return 0
    return numerator / (np.sqrt(denominator_x) * np.sqrt(denominator_y))


# Нормировка значений
def constraint(y_values):
    return np.sum(np.array(y_values) ** 2) - 1


# Начальные значения
y_initial = [0, centroid(x0, x1), centroid(x1, np.inf)]

y_initial = np.array(y_initial)
y_values = y_initial / np.sqrt(np.sum(y_initial**2))


# Оптимизация
result = minimize(
    lambda y: -correlation(y),
    y_initial,
    constraints={"type": "eq", "fun": constraint},
    method="SLSQP",
    options={"maxiter": 100, "disp": False},
)
optimal_y = result.x

# Вывод результатов
max_corr = -result.fun
print(f"{max_corr:.5f}")
print(f"{y_values[0]:.5f},{y_values[1]:.5f},{y_values[2]:.5f}")

ValueError: could not convert string to float: ''

### Task 7 Восстановить ядро свертки

In [12]:
import numpy as np


def recover_kernel(image, conv_result, kernel_size):
    # Размеры изображения и ядра
    m, n = image.shape
    h, w = kernel_size

    # Определяем отступы (паддинг)
    pad_h = (h - 1) // 2
    pad_w = (w - 1) // 2

    # Расширяем изображение нулями для учета паддинга
    padded_image = np.pad(
        image, ((pad_h, pad_h), (pad_w, pad_w)), mode="constant", constant_values=0
    )

    # Формируем систему уравнений
    A = []
    b = []
    for i in range(m):
        for j in range(n):
            # Извлекаем окно
            window = padded_image[i : i + h, j : j + w].flatten()
            A.append(window)
            b.append(conv_result[i, j])

    A = np.array(A)
    b = np.array(b)

    # Решаем систему линейных уравнений методом наименьших квадратов
    kernel_flat, _, _, _ = np.linalg.lstsq(A, b, rcond=None)

    # Преобразуем результат в матрицу ядра
    kernel = kernel_flat.reshape(h, w)
    return kernel


# Размер входной/выходной картинки
m, n = list(map(int, input().split(",")))

# Входное изображение в виде матрицы m x n
inp_image = []
for _ in range(m):
    inp_image.append(np.array(list(map(int, input().split(",")))))

inp_image = np.array(inp_image)

# Размер ядра свёртки, где h <= m, w <= n
h, w = list(map(int, input().split(",")))

# Выходное изображение в виде матрицы m x n
out_image = []
for _ in range(m):
    out_image.append(np.array(list(map(int, input().split(",")))))

out_image = np.array(out_image)

# Восстанавливаем ядро
kernel = np.round(recover_kernel(inp_image, out_image, (h, w))).astype(int)

for i in range(kernel.shape[0] - 1, -1, -1):
    print(*kernel[i,][::-1], sep=",")

ValueError: invalid literal for int() with base 10: ''

### Task 8 Обходы графа

In [13]:
import numpy as np


MOD = 1000000007


def matrix_mult(matrix, B, mod=MOD):
    n = len(matrix)
    C = [[0] * n for _ in range(n)]
    for i in range(n):
        for j in range(n):
            for k in range(n):
                C[i][j] = (C[i][j] + matrix[i][k] * B[k][j]) % mod
    return C


def matrix_power(matrix, power, mod=MOD):
    size = len(matrix)
    result = np.eye(size, dtype=np.int64)
    base = matrix.copy()

    while power > 0:
        if power % 2 == 1:
            result = matrix_mult(result, base, mod)
        base = matrix_mult(base, base, mod)
        power //= 2

    return result


def count_paths(adj_matrix, k):
    result_matrix = matrix_power(adj_matrix, k, MOD)
    return result_matrix[0][0]


n, k = list(map(int, input().split()))

adj_matrix = [list(map(int, input().split())) for _ in range(n)]

print(count_paths(adj_matrix, k))


ValueError: not enough values to unpack (expected 2, got 0)

### Task 9 Наклон переменной

In [ ]:
import numpy as np


def softmax(z):
    exp_z = np.exp(z - np.max(z))
    return exp_z / np.sum(exp_z, axis=1, keepdims=True)


def softmax_derivative(softmax_output):
    s = softmax_output.reshape(-1, 1)
    return np.diagflat(s) - np.dot(s, s.T)


def compute_gradient(m, Z, V):

    softmax_Z = softmax(Z)
    Y = softmax_Z @ V

    dL_dY = np.ones_like(Y)
    jacobian_softmax = softmax_derivative(softmax_Z[0])
    dL_dZ = (dL_dY @ V.T) @ jacobian_softmax

    return dL_dZ


m = int(input())
Z = np.array([list(map(float, input().split()))])
V = np.array([list(map(float, input().split())) for _ in range(m)])

dL_dZ = compute_gradient(m, Z, V)

print(" ".join(map(str, dL_dZ.flatten())))
